<a href="https://colab.research.google.com/github/Zen-Teng/Stylos/blob/main/cleaner_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read a PDF and print raw text to the console.

In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.4 MB/s eta 0:00:00


In [2]:
import pdfplumber

def read_pdf(file_path):
    text = ""

    with pdfplumber.open(file_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                text += f"\n--- Page {page_number} ---\n"
                text += page_text

    return text


In [3]:
from google.colab import files
uploaded = files.upload()

pdf_path = next(iter(uploaded))  # gets the uploaded filename
pdf_text = read_pdf(pdf_path)

print(pdf_text)

#For multiple upload
# for pdf_path in uploaded.keys():
#     print(f"Reading {pdf_path}")
#     pdf_text = read_pdf(pdf_path)
#     print(pdf_text)

Saving Eating disorder.pdf to Eating disorder.pdf

--- Page 1 ---
Eating disorders are a growing concern in today’s society, affecting people of all ages and
backgrounds. Driven by body image pressures, social media, and mental health challenges,
conditions like anorexia, bulimia, and binge-eating have serious physical and psychological
consequences. Awareness and early intervention are crucial to support recovery and
well-being.
The first dataset, Dataset_Eating_Disorder, highlights the risk among individuals
vulnerable to eating disorders. In this study, 106 out of 700 entries—approximately
15.1%—are considered at risk. The following graph shows the distribution of these
individuals who scored high (above 4 out of 5) on three indicators: mood influencing eating
patterns, worry about weight affecting eating patterns, and the duration of experiencing body
dysmorphia.
Building on the previous dataset, the second dataset,
Dataset_on_Eating_Habits_and_Mental_Health, provides further insig

Split that text into paragraphs. Save to data.json.

In [4]:
import re

#Method 1: Blank-line based (high confidence)
def split_by_blank_lines(text):
    paragraphs = re.split(r'\n\s*\n', text)
    return [p.strip() for p in paragraphs if p.strip()]


#Method 2: Wrapped-line reconstruction (most common)
def split_wrapped_lines(text):
    paragraphs = []
    current = []

    for line in text.splitlines():
        line = line.strip()

        if not line:
            if current:
                paragraphs.append(" ".join(current))
                current = []
        else:
            current.append(line)

    if current:
        paragraphs.append(" ".join(current))

    return paragraphs


#Method 3: Heuristic sentence-based (fallback)
def split_by_heuristic(text):
    text = re.sub(r'\n+', ' ', text)  # remove line breaks
    paragraphs = re.split(
        r'(?<=[.!?])\s+(?=[A-Z])',
        text
    )
    return [p.strip() for p in paragraphs]


In [11]:
#Step 1: Analyse the text
def analyze_text(text):
    lines = text.splitlines()

    empty_lines = sum(1 for l in lines if not l.strip())
    avg_line_length = sum(len(l) for l in lines) / max(len(lines), 1)

    return {
        "empty_lines": empty_lines,
        "avg_line_length": avg_line_length
    }


#Step 2: From the result in Step 1, determine which methods to use for paragraph identification
def split_paragraphs(text):
    stats = analyze_text(text)

    if stats["empty_lines"] > 2:
        print("Method used: split_by_blank_lines")
        return split_by_blank_lines(text)

    # if stats["avg_line_length"] < 80:
    #     print("Method used: split_wrapped_lines")
    #     return split_wrapped_lines(text)

    print("Method used: split_by_heuristic")
    return split_by_heuristic(text)


#Step 3: Sanity check: A threshold to see if the splitted paragraphs are of conventional length
def is_reasonable(paragraphs):
    if not paragraphs:
        return False
    avg_len = sum(len(p) for p in paragraphs) / len(paragraphs)
    return 50 < avg_len < 2000


In [13]:
text_stats = analyze_text(pdf_text)
print(text_stats)

paragraphs = split_paragraphs(pdf_text)
print(len(paragraphs))

is_reasonable(paragraphs)

print(paragraphs)

{'empty_lines': 1, 'avg_line_length': 70.0677966101695}
Method used: split_by_heuristic
24
['--- Page 1 --- Eating disorders are a growing concern in today’s society, affecting people of all ages and backgrounds.', 'Driven by body image pressures, social media, and mental health challenges, conditions like anorexia, bulimia, and binge-eating have serious physical and psychological consequences.', 'Awareness and early intervention are crucial to support recovery and well-being.', 'The first dataset, Dataset_Eating_Disorder, highlights the risk among individuals vulnerable to eating disorders.', 'In this study, 106 out of 700 entries—approximately 15.1%—are considered at risk.', 'The following graph shows the distribution of these individuals who scored high (above 4 out of 5) on three indicators: mood influencing eating patterns, worry about weight affecting eating patterns, and the duration of experiencing body dysmorphia.', 'Building on the previous dataset, the second dataset, Datase

todo:
- how to preserve table structure
- how to split into paragraphs
- research: how to extract wiritng style